### Import libraries

In [2]:
from  datetime import datetime, timedelta
import gc
import numpy as np, pandas as pd
import time

pd.options.display.max_columns = 50

### Download datasets from M5-competition guide

https://mofc.unic.ac.cy/m5-competition/

Describe categorical and price columns with its data types

In [23]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }

In [ ]:
h = 30 # forecast horizon
max_lags = 57 # maximum lags
tr_last = 1913 # last row number to be used for training 

Read calendar events data.

In [41]:
cal = pd.read_csv("calendar.csv")
cal.head()

,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,NaN,NaN,NaN,NaN,1,0,1


In [42]:
# create mapping for week ids
wk_ids = sorted(cal['wm_yr_wk'].unique())
week_id_dict = {}
for i,id in enumerate(wk_ids):
    week_id_dict[id] = i+1

Lets define function to get processed dataset

In [43]:
def create_dt(is_train = True, nrows = None, first_day = 1200):
    '''
    function to create preprocessed dataset from raw data to be
    compatible with TFT model.
    @is_train - is the dataset for training or nor, dtype:Boolean
    @nrows - number of rows of the raw dataset, dtype:integer
    @first_day - first day for the dataset, dtype:integer
    
    returns dt- dataframe of the preprocessed dataset
    '''
    prices = pd.read_csv("sell_prices.csv", dtype = PRICE_DTYPES)
    prices['wm_yr_wk'] = pd.to_numeric(prices['wm_yr_wk'].map(week_id_dict), downcast='integer')
    for col, col_dtype in PRICE_DTYPES.items():
        if col_dtype == "category":
            prices[col] = prices[col].cat.codes.astype("int16")
            prices[col] -= prices[col].min()
     
    # read calendar events data
    cal = pd.read_csv("calendar.csv", dtype = CAL_DTYPES)
    cal["date"] = pd.to_datetime(cal["date"])
    cal['wm_yr_wk']=pd.to_numeric(cal['wm_yr_wk'].map(week_id_dict), downcast='integer')
    
    for col, col_dtype in CAL_DTYPES.items():
        if col_dtype == "category":
            cal[col] = cal[col].cat.codes.astype("int16")
            cal[col] -= cal[col].min()
    
    # decide start day based on max_lags
    start_day = max(1 if is_train  else tr_last-max_lags, first_day)
    
    # create separate list for numerical and categorical columns
    numcols = [f"d_{day}" for day in range(start_day,tr_last+1)]
    catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
    
    # set datatypes for these columns (can be reduced after some analysis on data)
    dtype = {numcol:"float32" for numcol in numcols} 
    dtype.update({col: "category" for col in catcols if col != "id"})
    
    # read raw training data
    dt = pd.read_csv("sales_train.csv", \
                     nrows = nrows, usecols = catcols + numcols, dtype = dtype)

    for col in catcols:
        if col != "id":
            dt[col] = dt[col].cat.codes.astype("int16")
            dt[col] -= dt[col].min()
    
    if not is_train:
        for day in range(tr_last+1, tr_last+ 28 +1):
            dt[f"d_{day}"] = np.nan
    
    # melt this dataframe
    dt = pd.melt(dt,
                  id_vars = catcols,
                  value_vars = [col for col in dt.columns if col.startswith("d_")],
                  var_name = "d",
                  value_name = "sales")
    print(dt.shape)
    cal['d'] = ['d_'+str(i+1) for i in range(len(cal))]
    
    # simple join of the dataset with calendar events
    dt = dt.merge(cal, on= "d", copy = False)
    dt.drop('d', axis=1, inplace=True)
    print(dt.shape)
    
    # left join this dataset with price data
    dt = dt.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False, how="left")
    dt['sell_price'].fillna(0, inplace=True)
    
    # create a new column for day of the month
    dt.insert(8, 'day_of_month', '')
    
    dt['day_of_month'] = pd.to_numeric(dt['date'].dt.day, downcast='integer')
    print(dt.shape)

    dt.drop(['wday'], axis=1, inplace=True)
    return dt

Lets generate the dataset for training

In [44]:
# generate prprocessed dataset
processed_df = create_dt(is_train=True, first_day= 1)
processed_df.shape # shape of the new dataset

(58327370, 8)
(58327370, 20)
(58327370, 22)


(58327370, 21)

In [45]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58327370 entries, 0 to 58327369
Data columns (total 21 columns):
 #   Column        Dtype         
---  ------        -----         
 0   id            object        
 1   item_id       int16         
 2   dept_id       int16         
 3   store_id      int16         
 4   cat_id        int16         
 5   state_id      int16         
 6   sales         float32       
 7   date          datetime64[ns]
 8   day_of_month  int8          
 9   wm_yr_wk      int16         
 10  weekday       int16         
 11  month         int16         
 12  year          int16         
 13  event_name_1  int16         
 14  event_type_1  int16         
 15  event_name_2  int16         
 16  event_type_2  int16         
 17  snap_CA       float32       
 18  snap_TX       float32       
 19  snap_WI       float32       
 20  sell_price    float32       
dtypes: datetime64[ns](1), float32(5), int16(13), int8(1), object(1)
memory usage: 3.9+ GB


In [46]:
processed_df.isnull().sum()

id              0
item_id         0
dept_id         0
store_id        0
cat_id          0
state_id        0
sales           0
date            0
day_of_month    0
wm_yr_wk        0
weekday         0
month           0
year            0
event_name_1    0
event_type_1    0
event_name_2    0
event_type_2    0
snap_CA         0
snap_TX         0
snap_WI         0
sell_price      0
dtype: int64

In [32]:
processed_df.describe()

,item_id,dept_id,store_id,cat_id,state_id,sales,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
count,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07,4.602796e+07
mean,1.515382e+03,3.441378e+00,4.512308e+00,1.279055e+00,9.023523e-01,1.385844e+00,1.552365e+02,2.999338e+00,6.398704e+00,2.013552e+03,1.265242e+00,1.852089e-01,5.404824e-03,2.654517e-03,3.283675e-01,3.287932e-01,3.288327e-01,4.408089e+00
std,8.798533e+02,2.100407e+00,2.868218e+00,7.594233e-01,8.272933e-01,4.285680e+00,7.517313e+01,1.998840e+00,3.480230e+00,1.470835e+00,4.943580e+00,6.809139e-01,1.300769e-01,6.205678e-02,4.696193e-01,4.697747e-01,4.697891e-01,3.403657e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,2.011000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-02
25%,7.530000e+02,2.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,9.500000e+01,1.000000e+00,3.000000e+00,2.012000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.180000e+00
50%,1.503000e+03,3.000000e+00,5.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.630000e+02,3.000000e+00,6.000000e+00,2.014000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.470000e+00
75%,2.285000e+03,6.000000e+00,7.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,2.200000e+02,5.000000e+00,9.000000e+00,2.015000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.840000e+00
max,3.048000e+03,6.000000e+00,9.000000e+00,2.000000e+00,2.000000e+00,7.630000e+02,2.740000e+02,6.000000e+00,1.200000e+01,2.016000e+03,3.000000e+01,4.000000e+00,4.000000e+00,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.073200e+02


In [ ]:
print(processed_df.columns)

It would be better to zip it before downloading so we will use gzip compression for this purpose

In [47]:
start_time = time.time()
processed_df.to_csv("walmart.csv.gz", compression='gzip')
print("File compressed and saved in ",(time.time()-start_time)//60, "minutes ", (time.time()-start_time)%60, " seconds")

File compressed and saved in  46.0 minutes  1.0245311260223389  seconds


https://stackoverflow.com/questions/40660331/pandas-to-csv-slow-saving-large-dataframe

The final dataset would be large in size ~ 5GB.
### The original/raw data can still be reduced in terms of memory by logically downgrading the datatype for each column. This will significantly reduce the size of data.

### This considers preprocessing for only training data. Validation data should be preprocessed similarly. 